In [1]:
import time
import json
import requests
import random
from bs4 import BeautifulSoup

In [2]:
USER_AGENT = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}

**TASK 1**

In [3]:
def load_queries(filename):
    with open(filename, "r") as f:
        return [line.strip() for line in f.readlines()]

def search(engine, query):
    time.sleep(random.randint(10, 100))
    url = 'http://www.bing.com/search?q=' + "+".join(query.split())
    response = requests.get(url, headers=USER_AGENT)
    soup = BeautifulSoup(response.text, "html.parser")
    return scrape_results(soup, engine)

def scrape_results(soup, engine):
    raw_results = soup.find_all("li", attrs={"class": "b_algo"})

    results = []
    for result in raw_results[:10]:
        link = result.find("a")["href"] if result.find("a") else None
        if link and link not in results:
            results.append(link)
    return results

In [4]:
queries = load_queries("C:/Users/admin/Desktop/MS/Sem 3/IR/HW1/100QueriesSet1.txt")
results = {}
for query in queries:
    results[query] = search("Bing", query)  
with open("C:/Users/admin/Desktop/MS/Sem 3/IR/HW1/hw1.json", "w") as f:
    json.dump(results, f, indent=4)

**TASK 2**

In [47]:
import numpy as np
import pandas as pd
import json

bing_file_path = "C:/Users/admin/Desktop/MS/Sem 3/IR/HW1/hw1.json"
with open(bing_file_path, "r") as f:
    bing_results = json.load(f)

google_file_path = "C:/Users/admin/Desktop/MS/Sem 3/IR/HW1/Google_Result1.json"
with open(google_file_path, "r") as f:
    google_results = json.load(f)

In [48]:
def spearman_rank_correlation(google_urls, bing_urls):
    matched_urls = list(set(google_urls) & set(bing_urls))
    n = len(matched_urls)

    if n < 2:  # Fix: Directly check `n`
        return 0  

    # Create ranking dictionaries for Google and Bing
    google_ranks = {url: rank + 1 for rank, url in enumerate(google_urls)}
    bing_ranks = {url: rank + 1 for rank, url in enumerate(bing_urls)}

    # Extract ranks for matched URLs
    google_ranks_list = [google_ranks[url] for url in matched_urls]
    bing_ranks_list = [bing_ranks[url] for url in matched_urls]

    # Compute rank differences
    d_i_squared = [(g - b) ** 2 for g, b in zip(google_ranks_list, bing_ranks_list)]

    # Spearman formula
    sum_d_i_squared = sum(d_i_squared)
    rho = 1 - (6 * sum_d_i_squared) / (n * (n**2 - 1))

    return rho


In [49]:
final_results = []
for idx, query in enumerate(google_results.keys()):
    google_urls = google_results[query]
    bing_urls = bing_results.get(query, [])

    matching_urls = list(set(google_urls) & set(bing_urls))
    num_matching_urls = len(matching_urls)
    overlap_percentage = num_matching_urls / len(google_urls) * 100 if google_urls else 0

    spearman_coefficient = spearman_rank_correlation(google_urls, bing_urls)

    final_results.append([f"Query {idx+1}", num_matching_urls, overlap_percentage, spearman_coefficient])

In [50]:
df = pd.DataFrame(final_results, columns=["Query", "Number of Overlapping Results", "Percent Overlap", "Spearman Coefficient"])
df["Spearman Coefficient"] = df["Spearman Coefficient"].round(4)

average_values = df[["Number of Overlapping Results", "Percent Overlap", "Spearman Coefficient"]].mean()
df = df.append({
    "Query": "Averages",
    "Number of Overlapping Results": average_values["Number of Overlapping Results"],
    "Percent Overlap": average_values["Percent Overlap"],
    "Spearman Coefficient": average_values["Spearman Coefficient"]
}, ignore_index=True)
csv_file_path = "hw_1.csv"
df.to_csv(csv_file_path, index=False)

print(f"Comparison table saved as {csv_file_path}")

Comparison table saved as hw_1.csv
